# First Howemork. Optimization and Analytics

#### Done by Javier Alzuaz & Jaime Lobato

### Motivation for the Optimization problem: improving the Indiana Pacers’ performance through optimal lineup selection

The Indiana Pacers reached the NBA Finals last season, establishing themselves as one of the strongest teams in the league. However, their performance this year has significantly got worse, mainly due to a combination of injuries, inconsistent rotations, and the need to rely more heavily on G-League call-ups to maintain roster depth.

This sudden transition from contending for a championship to being in the last place in the NBA standings creates a realistic decision-making scenario for applying optimization techniques.

The coaching staff faces an important question:

Given the current roster performances, injuries, and the mix of NBA and G-League players, what starting lineup would maximize the team’s on-court effectiveness?

This is a strategic problem with limited resources (healthy players, budget) and clear performance metrics, making it ideal for formulation as a Linear Optimization and later a Mixed-Integer Optimization model.

In [1]:
import pandas as pd
from pyomo.environ import *
import numpy as np

In [2]:
data = pd.read_csv("pacers_stats.csv" ,sep =";", encoding='latin-1')

data

,Rk,Player,Age,Pos,G,GS,MP,FG,FGA,FG%,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards,Player-additional
0,1.0,Bennedict Mathurin,23.0,SF,2,2,365,85,155,0.548,...,55,70,25,0,0,25,30,310,NaN,mathube01
1,2.0,Pascal Siakam,31.0,PF,11,11,349,86,194,0.446,...,57,74,53,11,4,26,33,241,NaN,siakapa01
2,3.0,Aaron Nesmith,26.0,SF,11,11,305,49,134,0.367,...,29,45,15,8,3,7,25,155,NaN,nesmiaa01
3,4.0,Jarace Walker,22.0,PF,12,7,291,34,115,0.297,...,44,52,33,4,6,21,21,103,NaN,walkeja02
4,5.0,Andrew Nembhard,26.0,PG,5,5,274,52,142,0.366,...,12,14,68,4,0,22,26,172,NaN,nembhan01
5,6.0,Obi Toppin,27.0,PF,3,0,273,50,120,0.417,...,57,67,17,10,0,20,23,140,NaN,toppiob01
6,7.0,Ben Sheppard,24.0,SG,12,5,254,25,78,0.323,...,38,47,17,4,2,7,26,68,NaN,sheppbe01
7,8.0,Quenton Jackson,27.0,PG,5,3,202,40,74,0.541,...,24,34,36,10,2,12,20,118,NaN,jacksqu01
8,9.0,James Wiseman,24.0,C,1,1,200,20,30,0.667,...,0,40,0,0,10,30,20,40,NaN,wisemja01
9,10.0,Jeremiah Robinson-Earl,25.0,PF,7,2,194,20,56,0.359,...,34,64,10,7,0,3,11,54,NaN,robinje02


In [8]:
data["Pos"] = data["Pos"].fillna("").astype(str)
data['valoration']= data['FG%']+data['FT%']+data['3P%']+data['PTS']+data['STL']+data['TRB']+data['AST']+data['BLK']-data['TOV']-data['PF']
data

,Rk,Player,Age,Pos,G,GS,MP,FG,FGA,FG%,...,TRB,AST,STL,BLK,TOV,PF,PTS,Awards,Player-additional,valoration
0,1.0,Bennedict Mathurin,23.0,SF,2,2,365,85,155,0.548,...,70,25,0,0,25,30,310,NaN,mathube01,351.933
1,2.0,Pascal Siakam,31.0,PF,11,11,349,86,194,0.446,...,74,53,11,4,26,33,241,NaN,siakapa01,325.423
2,3.0,Aaron Nesmith,26.0,SF,11,11,305,49,134,0.367,...,45,15,8,3,7,25,155,NaN,nesmiaa01,195.535
3,4.0,Jarace Walker,22.0,PF,12,7,291,34,115,0.297,...,52,33,4,6,21,21,103,NaN,walkeja02,157.402
4,5.0,Andrew Nembhard,26.0,PG,5,5,274,52,142,0.366,...,14,68,4,0,22,26,172,NaN,nembhan01,211.579
5,6.0,Obi Toppin,27.0,PF,3,0,273,50,120,0.417,...,67,17,10,0,20,23,140,NaN,toppiob01,192.593
6,7.0,Ben Sheppard,24.0,SG,12,5,254,25,78,0.323,...,47,17,4,2,7,26,68,NaN,sheppbe01,106.327
7,8.0,Quenton Jackson,27.0,PG,5,3,202,40,74,0.541,...,34,36,10,2,12,20,118,NaN,jacksqu01,169.763
8,9.0,James Wiseman,24.0,C,1,1,200,20,30,0.667,...,40,0,0,10,30,20,40,NaN,wisemja01,NaN
9,10.0,Jeremiah Robinson-Earl,25.0,PF,7,2,194,20,56,0.359,...,64,10,7,0,3,11,54,NaN,robinje02,122.430


In [9]:
players_idx = list(data.index)

PG_idx = [i for i in players_idx if "PG" in data.loc[i, "Pos"]]
SG_idx = [i for i in players_idx if "SG" in data.loc[i, "Pos"]]
C_idx  = [i for i in players_idx if data.loc[i, "Pos"] == "C" or " C" in data.loc[i,"Pos"]]
F_idx  = [i for i in players_idx if any(p in data.loc[i, "Pos"] for p in ["SF", "PF"])]

# Diccionario de valoraciones
valoration_dict = data["valoration"].to_dict()

In [24]:
from pyomo.environ import *

import pandas as pd
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory, NonNegativeReals, maximize
from itertools import combinations  
model = ConcreteModel()
model.PLAYERS = Set(initialize=players_idx)

model.valoration = Param(
    model.PLAYERS,
    initialize=valoration_dict
)

model.x = Var(model.PLAYERS, domain=Binary)

# Conjunto de jugadores
"""model.PLAYERS = Set(initialize=players_idx)

# Parámetro: valoración de cada jugador
model.valoration = Param(
    model.PLAYERS,
    initialize=valoration_dict,
    within=float
)"""

"""# Variable binaria: x[i] = 1 si el jugador i está en el quinteto
model.x = Var(model.PLAYERS, domain=Binary)"""


'# Variable binaria: x[i] = 1 si el jugador i está en el quinteto\nmodel.x = Var(model.PLAYERS, domain=Binary)'

In [25]:
def objective_rule(m):
    return sum(m.valoration[i] * m.x[i] for i in m.PLAYERS)

model.OBJ = Objective(rule=objective_rule, sense=maximize)
def lineup_size_rule(m):
    return sum(m.x[i] for i in m.PLAYERS) == 5

model.LineupSize = Constraint(rule=lineup_size_rule)

# 5.2. Al menos 1 base (PG)
def pg_rule(m):
    return sum(m.x[i] for i in PG_idx) >= 1

model.MinPG = Constraint(rule=pg_rule)

# 5.3. Al menos 1 escolta (SG)
def sg_rule(m):
    return sum(m.x[i] for i in SG_idx) >= 1

model.MinSG = Constraint(rule=sg_rule)

# 5.4. Al menos 1 pívot (C)
def c_rule(m):
    return sum(m.x[i] for i in C_idx) >= 1

model.MinC = Constraint(rule=c_rule)

# 5.5. Al menos 2 aleros/interiores (SF o PF)
def f_rule(m):
    return sum(m.x[i] for i in F_idx) >= 2

model.MinF = Constraint(rule=f_rule)


In [26]:
possible_solvers = ["glpk", "cbc", "gurobi", "cplex"]
solver = None
solver_name = None

for name in possible_solvers:
    try:
        s = SolverFactory(name)
        if s is not None and s.available():
            solver = s
            solver_name = name
            break
    except:
        continue

use_pyomo_solution = False
results = None

if solver is None:
    print("\n⚠️  No se encontró ningún solver MILP instalado (glpk, cbc, gurobi, cplex).")
else:
    print(f"\nUsando solver: {solver_name}")
    try:
        results = solver.solve(model, tee=False)
        tc = results.solver.termination_condition
        print("Termination condition:", tc)

        if tc in [TerminationCondition.optimal, TerminationCondition.feasible]:
            use_pyomo_solution = True
        else:
            print(f"⚠️ Solver {solver_name} no terminó de forma óptima/viable. Se usará búsqueda por fuerza bruta.")
    except Exception as e:
        print(f"⚠️ Error al ejecutar el solver {solver_name}: {e}")
        print("Se usará búsqueda por fuerza bruta.")



Usando solver: glpk
ERROR: Solver (glpk) returned non-zero return code (1)
ERROR: Solver log: GLPSOL--GLPK LP/MIP Solver 5.0 Parameter(s) specified in
the command line:
     --write C:\Users\jaime\AppData\Local\Temp\tmp8mogqo2h.glpk.raw --wglp
     C:\Users\jaime\AppData\Local\Temp\tmpytiddch2.glpk.glp --cpxlp
     C:\Users\jaime\AppData\Local\Temp\tmpol5shk5n.pyomo.lp
    Reading problem data from
    'C:\Users\jaime\AppData\Local\Temp\tmpol5shk5n.pyomo.lp'...
    C:\Users\jaime\AppData\Local\Temp\tmpol5shk5n.pyomo.lp:13: constraints
    section missing CPLEX LP file processing error
⚠️ Error al ejecutar el solver glpk: Solver (glpk) did not exit normally
Se usará búsqueda por fuerza bruta.


In [28]:
if use_pyomo_solution:
    selected_idx = [i for i in model.PLAYERS if value(model.x[i]) > 0.5]
    method_used = "Pyomo (MILP)"
else:
    print("\n▶ Ejecutando búsqueda por fuerza bruta en todas las combinaciones de 5 jugadores...")
    best_score = -np.inf
    best_combo = None

    for combo in combinations(players_idx, 5):
        subset = data.loc[list(combo)]
        positions = subset["Pos"]

        n_pg = (positions.str.contains("PG")).sum()
        n_sg = (positions.str.contains("SG")).sum()
        n_c  = (positions.str.contains("C")).sum()
        n_f  = (positions.str.contains("SF") | positions.str.contains("PF")).sum()

        if not (n_pg >= 1 and n_sg >= 1 and n_c >= 1 and n_f >= 2):
            continue

        total_val = subset["valoration"].sum()
        if total_val > best_score:
            best_score = total_val
            best_combo = combo

    selected_idx = list(best_combo)
    method_used = "búsqueda exhaustiva en Python"

# ---------------------------------------------------------
# 7. Mostrar quinteto óptimo
# ---------------------------------------------------------
best_lineup = data.loc[selected_idx].copy()
total_val = best_lineup["valoration"].sum()

print(f"\nMejor quinteto encontrado usando {method_used}")
print(f"Valoration total = {total_val:.2f}\n")

print(
    best_lineup[
        ["Player", "Pos", "PTS", "TRB", "AST", "STL", "BLK", "TOV", "PF", "valoration"]
    ].to_string(index=False)
)

print("\nDistribución de posiciones en el quinteto:")
print(best_lineup["Pos"].value_counts())


▶ Ejecutando búsqueda por fuerza bruta en todas las combinaciones de 5 jugadores...

Mejor quinteto encontrado usando búsqueda exhaustiva en Python
Valoration total = 1083.57

            Player Pos  PTS  TRB  AST  STL  BLK  TOV  PF  valoration
Bennedict Mathurin  SF  310   70   25    0    0   25  30     351.933
     Pascal Siakam  PF  241   74   53   11    4   26  33     325.423
   Andrew Nembhard  PG  172   14   68    4    0   22  26     211.579
      Ben Sheppard  SG   68   47   17    4    2    7  26     106.327
          Jay Huff   C   53   31    8    1   19    6  19      88.309

Distribución de posiciones en el quinteto:
Pos
SF    1
PF    1
PG    1
SG    1
C     1
Name: count, dtype: int64
